Les importations

In [1]:
import matplotlib.pyplot as plt
import numpy as np

Création d'un damier

In [2]:
try:
    tailleDamier = int(input('Quel est la taille de votre damier choisis ?'))
except:
    print('Rentrer un entier positif')
    
xx, yy = np.meshgrid(np.zeros(tailleDamier),np.zeros(tailleDamier))

damier = xx

#print(damier)

Quel est la taille de votre damier choisis ?10


Création de la Narrow Band

In [3]:
def voisins(damier, case):
    i = case[0]
    j = case[1]
    return [(a,b) for (a,b) in [(i-1,j),  (i, j+1), (i+1,j), (i,j-1)] if (a in range(len(damier)) and b in range(len(damier[0])) and (damier[a][b]==0 or damier[a][b]==np.inf))]

In [4]:
try:
    caseFigeeX = (int(input('Quel case voulez-vous figée (coordonnée en x) ?')) - 1)
    caseFigeeY = (int(input('Quel case voulez-vous figée (coordonnée en y) ?')) - 1) 
except:
    print('Mettre un entier positif compris entre 1 et la taille du damier')
    
origine = caseFigeeX, caseFigeeY

narrowBand = np.array(voisins(damier, origine))

#print(narrowBand)

Quel case voulez-vous figée (coordonnée en x) ?4
Quel case voulez-vous figée (coordonnée en y) ?0


Création de la notion de distance dans ce damier

In [5]:
h = float(input('Définir une longeur de cellule en mètres : '))

def distance(origine, case):
    listeDistance = []
    if type(case[0]) == np.int64 or type(case[0]) == int :
        return np.sqrt((h*case[0]-h*origine[0])**2 + (h*case[1]-h*origine[1])**2 )
    
    for i in range(len(case)):
        listeDistance.append(np.sqrt((h*case[i][0]-h*origine[0])**2 + (h*case[i][1]-h*origine[1])**2 ))
    return listeDistance

#print(distance([0,0],[2,2]))
#print(distance([0,0],[[2,2],[3,3],[5,5]]))

Définir une longeur de cellule en mètres : 1


Fonction du gradient

In [6]:
def gradX (origine, caseFigee, h, tailleDamier ):
    dCaseFigee = distance(origine, caseFigee)
    
    if caseFigee[0]<tailleDamier-1  :
        dCaseFigeeDroite = distance(origine, [caseFigee[0]+1, caseFigee[1]] )        
    else:
        dCaseFigeeDroite=np.inf
        
    if caseFigee[0]>0:    
        dCaseFigeeGauche = distance(origine, [caseFigee[0]-1, caseFigee[1]])
    else:
        dCaseFigeeGauche=np.inf
    
    dMinimalGradX = np.min([dCaseFigeeDroite, dCaseFigeeGauche])
    return (dCaseFigee-dMinimalGradX)/h
    
def gradY (origine, caseFigee, h, tailleDamier):
    dCaseFigee = distance(origine, caseFigee)
    
    if caseFigee[1]<tailleDamier-1:
        dCaseFigeeHaut = distance(origine, [caseFigee[0], caseFigee[1]+1] )
    else:
        dCaseFigeeHaut = np.inf
        
    if caseFigee[1]>0:
        dCaseFigeeBas = distance(origine, [caseFigee[0], caseFigee[1]-1])
    else:
        dCaseFigeeBas = np.inf
    
    dMinimalGradY = np.min([dCaseFigeeBas, dCaseFigeeHaut])
    return (dCaseFigee-dMinimalGradY)/h

Répends la vague

In [7]:
while len(narrowBand)!=0:
    #chope la case la plus proche dans le narrowBand et on l'appelle nouvelleCaseFigee
    narrowBandDistance = distance(origine, narrowBand)
    indiceMinimalDistance = list(narrowBandDistance).index(np.min(narrowBandDistance))
    caseFigee = narrowBand[indiceMinimalDistance]
    
    #on prend le gradient de la nouvelle case figée
    gradDx = gradX(origine, caseFigee, h, tailleDamier)
    gradDy = gradY(origine, caseFigee, h, tailleDamier)
    
    
    #applique la formule de 1/F
    damier[caseFigee[0],caseFigee[1]] = np.sqrt(gradDx**2 + gradDy**2)
    
    
    #ajoute les nouveaux voisins à la narrowBand
    nouveauxVoisins = np.array([[a,b] for (a,b) in voisins(damier,caseFigee) if ([a,b]!=[origine[0],origine[1]])   ] )
    if len(nouveauxVoisins)>0:
        narrowBand = np.concatenate((narrowBand,nouveauxVoisins))
    
    #supprime la case qui a été figée et les doublons dans la narrowBand
    narrowBand = np.delete(narrowBand, indiceMinimalDistance, 0)
    narrowBand = np.unique(narrowBand, axis=0)
    

    #print('\n', narrowBand)

#print(damier)

ValueError: cannot reshape array of size 0 into shape (0,newaxis)

Faisons en un beau graphe

In [ ]:
fig = plt.figure()
im = plt.imshow(damier, origin='lower',extent=[0,tailleDamier,0,tailleDamier])
plt.xlabel('longueur')
plt.ylabel('largeur')
c = fig.colorbar(im)
c.set_label('1/F')
plt.show()